In [16]:
import pickle
import gzip

#how be able to iterate through all of the different files

#updated
vals_dict = {'tfs':[0.25, 0.75, 0.9, 0.95, 0.99],
'n': [0.5, 0.63, 0.69, 0.81, 0.75, 0.9], 'k':[1,40,200]  }
'''{'tfs':[0.25, 0.75, 0.9, 0.95, 0.99], 'flat':[0.01, 0.02, 0.05],
'n': [0.1, 0.25, 0.5, 0.63, 0.69, 0.75, 0.81, 0.9], 'k':[1,10,40,200]  }'''



"{'tfs':[0.25, 0.75, 0.9, 0.95, 0.99], 'flat':[0.01, 0.02, 0.05],\n'n': [0.1, 0.25, 0.5, 0.63, 0.69, 0.75, 0.81, 0.9], 'k':[1,10,40,200]  }"

In [17]:
#stat sig:
import numpy as np
vals = [1]*13
vals += [0]*5
tot  = len(vals)
vals = np.asarray(vals)
print('mean',vals.mean(), 'stand err',(1.96*(vals.std()/np.sqrt(tot))))
print('mean',vals.mean()-(1.96*(vals.std()/np.sqrt(tot))))

mean 0.7222222222222222 stand err 0.2069207252957695
mean 0.5153014969264527


In [18]:
import numpy as np 
import pandas as pd
prompts=pd.read_csv('test_dataframe_500primer.csv') # the text has the prompts so dont need this. 

In [19]:
gpt_out_path = 'gpt-2_output/'
additional_path = '-model_774M-seed_27'
text = pickle.load( gzip.open(gpt_out_path+'all_text_perplexity_scores_for_the_dataset_Human_StoryPrompts_Completion.csv'+additional_path+'.pickle.gz', 'rb'))


# Convert all of the numbers text from all of the different models into one dataframe with all of the prompts and the produced text separated out

In [20]:
# used to decode:

from decodeLogits import *
batch = 0
ind = 0
'''
tokens = []
for j in range(text[0].shape[2]):
    tokens.append(np.argmax(df[0][ind, :, j]))'''
decoder_text(text[batch][ind])

"[WP] You look out at the bright lights of the city and wonder how you ever grew up. \n The lights dance across the evening sky. You can see the buildings in the distance, some lights turning of as the stores close down. You wonder how this all happened, when did everything change? \n \n When did *you* change? \n \n Everyone is born a child. Some grow up faster than others. Some do n't grow up at all. Hours of playing. Even school was fun. Learning games and imaginary friends, but that stuff is boring now. A little older and sports are the thing. Running around the field, ball between your legs, trying not to trip as you kick for the goal. Does n't matter much if you miss, you're having fun with friends. Sure, they may laugh if you miss, but you laugh right along with them. \n \n A glance at the body next to you. So calm, so quiet. Were you ever like that? It's hard to remember. You feel the knife in your hand. It feels heavier now than normal, as if this moment is more important than 

## Generating all completions for a particular random prompt

In [21]:
seed = 27
prompt_length = 100
gs=dict()
methods_wanted = ['tfs_0.95', 'n_0.69']#, 'k_40']
additional_path = '774M_model'
first = True
for key, params in vals_dict.items():
    for par in params:
        if key+'_'+str(par) in methods_wanted:
            #all_logits = pickle.load( gzip.open('gpt-2_output/all_logits_'+key+'-sampling-type_'+par+'-sampling-param_100-word-prompts_150-gen-length_100-number-of-prompts.pickle.gz', 'rb'))
            text = pickle.load( gzip.open('gpt-2_output/all_text_'+key+'-sampling-type_'+str(par)+'-sampling-param_100-word-prompts_150-gen-length_100-number-of-prompts_'+str(seed)+'-seed_'+additional_path+'.pickle.gz', 'rb'))
            rand_selects = pickle.load( gzip.open('gpt-2_output/prompt_rand_selections_'+key+'-sampling-type_'+str(par)+'-sampling-param_100-word-prompts_150-gen-length_100-number-of-prompts_'+str(seed)+'-seed_'+additional_path+'.pickle.gz', 'rb'))

            generated_output = []
            for batch in text:
                for entry in batch:
                    generated_output.append(decoder_text(entry[prompt_length:]))
            col_name = 'Samp Type:'+key+' Param:'+str(par)
            gs[col_name] = generated_output
            
    if first==True:
        prompts_used = []
        for batch in text:
            for entry in batch:
                prompts_used.append(decoder_text(entry[0:prompt_length]))
        
        first=False
gs = pd.DataFrame(gs)

In [22]:
gs.shape

(100, 2)

In [23]:
gs.head()

Samp Type:tfs Param:0.95  \
0   pass, and your time is up. You are one of the...   
1  n't I make sure he doesn't come back? \n \n \n...   
2   a lamp. So he was forced to create one. \n \n...   
3   we have her. She's now with us.''\n ``Oh, it'...   
4  , was a complete fucking idiot, who could neve...   

                              Samp Type:n Param:0.69  
0  , days, weeks, months, years. The life you liv...  
1   I try to tell it not to worry? \n \n \n This ...  
2   himself. He wished for the boy's family.\nAnd...  
3   are holding her hostage. We'll have her for r...  
4  , didn ’ t quite fit the character, and didn ’...

In [24]:
gs.iloc[0,0]

' pass, and your time is up. You are one of the last people to leave the park, and the last one to see the city. You take your last breath, and watch as the night closes in. \n You are in your house, and your room is dark. The lights are on, and your computer is on, but you are not even sure what time it is. You sit up, and see the light from the front door come on, and your heart leaps. \n You close your eyes as you hear the phone ring. You hear the voicemail. You pick it up, and the caller says "Hi, I\'m your brother. \xa0I\'m home. \xa0I\'m sorry for the long wait'

In [25]:
gs.iloc[3]

Samp Type:tfs Param:0.95     we have her. She's now with us.''\n ``Oh, it'...
Samp Type:n Param:0.69       are holding her hostage. We'll have her for r...
Name: 3, dtype: object

In [26]:
for ind, p in enumerate(prompts_used):
    if 'Stockholm syndrome, but the other way around' in p:
        print(ind)

3


# Mass generator.

In [27]:
import numpy as np


In [28]:
num_gens = 10
file_name_base = 'test_big'
answer_keys = []
with open(file_name_base + '_blind_output.txt', 'w') as file: 
    
    for ind in range(num_gens):
    
        select_a_random_prompt = np.random.choice(gs.shape[0])
        select_random_generation_methods = np.random.choice(range(gs.shape[1]), size =gs.shape[1], replace=False )
        #select_random_generation_methods = np.insert(select_random_generation_methods, 0, 0)
        print(select_a_random_prompt)
        select_random_generation_methods;
        
        answer_keys.append(gs.iloc[select_a_random_prompt,select_random_generation_methods].index.values)

        file.write('Prompt: \n \n')
        file.write(prompts_used[select_a_random_prompt])
        file.write('\n')
        file.write('================== \n')
        for ind, out in enumerate(gs.iloc[select_a_random_prompt,select_random_generation_methods].tolist()):
            file.write('Random Generation: '+str(ind)+' \n \n')
            file.write(out)
            print(out)

            file.write('\n')
            file.write('================== \n')
            if ind == gs.shape[1]-1: # dont write random generation at the end. 
                continue
                
        file.write('\n \n')
        file.write('=====================================================')
        file.write('\n \n')
        
with open(file_name_base + '_answers.txt', 'w') as file: 
    for ak in answer_keys:
        for ind, k in enumerate(ak):
            file.write(str(ind)+' : '+k+' \n')
        file.write('\n')
        file.write('=====================================================')
        file.write('\n')
pickle.dump(answer_keys,open(file_name_base + '_answers_list.pickle','wb'))

10
 so she can't even tell you her first name. It's like she's a mother's worst nightmare.
How to get rid of a mother's greatest asset? Step on a crack, break a mother's back  #proud to be a mother
You can get a new one, too.
When women say they're a "whore" they mean that they have no self-control. What they mean is that they're all about sex and they'll do anything to get it. But the women who say they're a whore are always talking about one or two things:
A) To get off
B) To make sure they're being watched
C) To have an orgasm
When you see a whore
 where she's constantly exhausted and never really has much time to sleep. A single tear in a mother's eyes is worth a lifetime of misery.
Posted by Mark Liberman at 10:26 AM<|endoftext|>It is the year 2049.

As the United States military and the CIA attempt to create a "New World Order," a terrorist organization known as the Valkyries is leading the charge. With the help of a mysterious woman, the Valkyries are leading a mysterious rebell

Some of the very tight ones are too close and degenerate into repeats. The prompts and stochasticity of a single generation are really hard to deal with though. it is also a lot to read in order to evaluate the quality. 
What is something I can evaluate algorithmically and where diverse beam search is outperformed? It may inherently need to be something that is long. 
Seems like there is definitely a sweet spot where dont want too many options but also dont want it to be degenerate. 

In [54]:
answer_keys

[array(['Samp Type:n Param:0.69', 'Samp Type:tfs Param:0.95'], dtype=object),
 array(['Samp Type:tfs Param:0.95', 'Samp Type:n Param:0.69'], dtype=object),
 array(['Samp Type:tfs Param:0.95', 'Samp Type:n Param:0.69'], dtype=object),
 array(['Samp Type:tfs Param:0.95', 'Samp Type:n Param:0.69'], dtype=object),
 array(['Samp Type:tfs Param:0.95', 'Samp Type:n Param:0.69'], dtype=object),
 array(['Samp Type:tfs Param:0.95', 'Samp Type:n Param:0.69'], dtype=object),
 array(['Samp Type:n Param:0.69', 'Samp Type:tfs Param:0.95'], dtype=object),
 array(['Samp Type:n Param:0.69', 'Samp Type:tfs Param:0.95'], dtype=object),
 array(['Samp Type:tfs Param:0.95', 'Samp Type:n Param:0.69'], dtype=object),
 array(['Samp Type:n Param:0.69', 'Samp Type:tfs Param:0.95'], dtype=object)]

In [55]:
# assessing:


alex_answers = [1, 0.5, 0.5, 0, 0, 0, 1, 0.5, 0, 1] # liked generation 0
alex_chose = []

for ind, a in enumerate(alex_answers):
    if a==0.5:
        continue
    elif a==1:
        want_key_ind = 0
    elif a==0:
        want_key_ind = 1
    chosen = answer_keys[ind][want_key_ind]
    alex_chose.append(chosen)

In [57]:
answer_keys

[array(['Samp Type:n Param:0.69', 'Samp Type:tfs Param:0.95'], dtype=object),
 array(['Samp Type:tfs Param:0.95', 'Samp Type:n Param:0.69'], dtype=object),
 array(['Samp Type:tfs Param:0.95', 'Samp Type:n Param:0.69'], dtype=object),
 array(['Samp Type:tfs Param:0.95', 'Samp Type:n Param:0.69'], dtype=object),
 array(['Samp Type:tfs Param:0.95', 'Samp Type:n Param:0.69'], dtype=object),
 array(['Samp Type:tfs Param:0.95', 'Samp Type:n Param:0.69'], dtype=object),
 array(['Samp Type:n Param:0.69', 'Samp Type:tfs Param:0.95'], dtype=object),
 array(['Samp Type:n Param:0.69', 'Samp Type:tfs Param:0.95'], dtype=object),
 array(['Samp Type:tfs Param:0.95', 'Samp Type:n Param:0.69'], dtype=object),
 array(['Samp Type:n Param:0.69', 'Samp Type:tfs Param:0.95'], dtype=object)]

In [56]:
alex_chose

['Samp Type:n Param:0.69',
 'Samp Type:n Param:0.69',
 'Samp Type:n Param:0.69',
 'Samp Type:n Param:0.69',
 'Samp Type:n Param:0.69',
 'Samp Type:n Param:0.69',
 'Samp Type:n Param:0.69']

In [49]:
gs.iloc[select_a_random_prompt,select_random_generation_methods]

array(['Samp Type:n Param:0.69', 'Samp Type:tfs Param:0.95'], dtype=object)

In [ ]:
gs.iloc[select_a_random_prompt,0]

In [ ]:
choose_prompt =rand_selects[select_a_random_prompt]
choose_prompt

In [ ]:
prompts.head()

In [ ]:
print(prompts.iloc[choose_prompt].test_target)